## **Setup the Workspace**

In [4]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
tf.__version__

'2.16.1'

## **Data Preprocessing**

### Generating images for the Training Dataset

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

### Generating Images for The Test Dataset 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

### Creating the Training Set (Reader File)

In [ ]:
training_set = train_datagen.flow_from_directory('', 
                                                target_size=(128, 128),
                                                batch_size = 10, 
                                                color_mode = 'grayscale',
                                                class_mode = 'categorical')

### Creating the Test Set (Reader File)

In [ ]:
testing_set = test_datagen.flow_from_directory('', 
                                                target_size=(128, 128),
                                                batch_size = 10, 
                                                color_mode = 'grayscale',
                                                class_mode = 'categorical')

## **Building CNN**

### Inisialization CNN 

In [15]:
classifier = tf.keras.models.Sequential()

### Convolution Layer

In [20]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                    kernel_size=3,
                                    padding="same",
                                    activation="relu",
                                    input_dim=[128, 128, 1]))

d:\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


### Pooling Layer

In [19]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                        strides=2,
                                        padding='valid'))

### Adding a second Convolutional Layer

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                    kernel_size=3, 
                                    padding="same",
                                    activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=3,
                                        strides=2,
                                        padding="valid"))

### Flattening

In [ ]:
classifier.add(tf.keras.layer.Flatten())

### Fully Connected Layer

In [ ]:
classifier.add(tf.keras.layer.Dense(units=128,
                                    activation='relu'))
classifier.add(tf.keras.layer.Dropout(0.50))
classifier.add(tf.keras.layer.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layer.Dropout(0.50))
classifier.add(tf.keras.layer.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layer.Dense(units=27, activation='softmax'))

## **Training CNN**

### Compiling the CNN Models

In [ ]:
classifier.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

### Summary The Models

In [ ]:
classifier.summary()

In [ ]:
classifier.fit(training_set,
                epochs=5,
                validation_data = test_set)

### Saving Models

In [ ]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print("Model Saved")

classifier.save_weight("model_new.h5")
print("Weight Saved")